## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-18-51-09 +0000,nyt,"Snubbing RFK Jr., States Announce Plans to Coo...",https://www.nytimes.com/2025/09/03/us/rfk-jr-v...
1,2025-09-03-18-44-56 +0000,nyt,F.D.A. Official Overruled Scientists on Wide A...,https://www.nytimes.com/2025/09/03/health/fda-...
2,2025-09-03-18-40-29 +0000,startribune,Live: Vice President JD Vance visits Minnesota...,https://www.startribune.com/jd-vance-mn-annunc...
3,2025-09-03-18-39-32 +0000,nyt,What Is Swedish Culture? IKEA? Yes. Abba? Not ...,https://www.nytimes.com/2025/09/03/arts/music/...
4,2025-09-03-18-38-00 +0000,wsj,New Orleans might be the next city to get addi...,https://www.wsj.com/politics/policy/trump-mull...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,55
220,china,26
126,putin,21
31,new,17
473,kim,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
212,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...,199
62,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...,155
157,2025-09-03-11-23-51 +0000,nyt,Xi Parades Military Strength as Trump Accuses ...,https://www.nytimes.com/2025/09/02/world/asia/...,139
190,2025-09-03-08-26-24 +0000,nyt,"China’s Military Parade, in Photos: Xi Unveils...",https://www.nytimes.com/2025/09/03/world/asia/...,138
228,2025-09-03-02-57-51 +0000,nypost,China’s Xi hosts largest-ever military parade ...,https://nypost.com/2025/09/02/world-news/china...,136


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
212,199,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...
267,61,2025-09-02-23-16-19 +0000,nypost,Dramatic footage shows US strike that oblitera...,https://nypost.com/2025/09/02/world-news/foota...
4,57,2025-09-03-18-38-00 +0000,wsj,New Orleans might be the next city to get addi...,https://www.wsj.com/politics/policy/trump-mull...
44,51,2025-09-03-17-16-24 +0000,nypost,"Eric Adams, White House in talks over Trump ad...",https://nypost.com/2025/09/03/us-news/mayor-er...
210,48,2025-09-03-05-23-07 +0000,nypost,Vice President JD Vance and second lady Usha w...,https://nypost.com/2025/09/03/us-news/vice-pre...
120,39,2025-09-03-14-20-52 +0000,latimes,Wildfire burns into historic California Gold R...,https://www.latimes.com/california/story/2025-...
280,38,2025-09-02-22-18-35 +0000,nypost,Six candidates from Germany’s right-wing AfD p...,https://nypost.com/2025/09/02/world-news/six-c...
43,37,2025-09-03-17-16-35 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...
32,34,2025-09-03-17-32-39 +0000,latimes,'Ketamine queen' pleads guilty in federal cour...,https://www.latimes.com/california/story/2025-...
62,34,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
